# Adaptative Guide-net simple

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [1]:
from time import time
import os
import gzip
import numpy as np
from keras.utils import np_utils

import matplotlib.pyplot as plt

Using TensorFlow backend.


In [ ]:
def load_mnist(path, kind='train'):
    import os
    import gzip
    import numpy as np

    """Load MNIST data from `path`"""
    labels_path = os.path.join(path,
                               '%s-labels-idx1-ubyte.gz'
                               % kind)
    images_path = os.path.join(path,
                               '%s-images-idx3-ubyte.gz'
                               % kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8,
                               offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8,
                               offset=16).reshape(len(labels), 784)

    return images, labels

In [ ]:
X_train, y_train = load_mnist('drive/TCC-ITAU/fashion-mnist/data/fashion', kind='train')
X_test, y_test = load_mnist('drive/TCC-ITAU/fashion-mnist/data/fashion', kind='t10k')

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1974)

In [ ]:
train_features = X_train.reshape(X_train.shape[0], 28, 28, 1)/255
vali_features = X_val.reshape(X_val.shape[0], 28, 28, 1)/255
test_features = X_test.reshape(X_test.shape[0], 28, 28, 1)/255

y_train_cat = np_utils.to_categorical(y_train)
y_val_cat = np_utils.to_categorical(y_val)
y_test_cat = np_utils.to_categorical(y_test)

## Coarse and Fine classe

| Label | Fine | Coarse |
| --- | --- | --- |
| 0 | T-shirt/top | TOP |
| 1 | Trouser | BOTTOM |
| 2 | Pullover | TOP |
| 3 | Dress |BODY |
| 4 | Coat | TOP |
| 5 | Sandal | FEET |
| 6 | Shirt | TOP |
| 7 | Sneaker | FEET |
| 8 | Bag | ACCESSORY |
| 9 | Ankle boot | FEET |

In [ ]:
dict_coarse = {0: 0,
              1: 1,
              2: 0,
              3: 2,
              4: 0,
              5: 3,
              6: 0,
              7: 3,
              8: 4,
              9: 3}

y_train_coarse = np.vectorize(dict_coarse.get)(y_train)
y_val_coarse = np.vectorize(dict_coarse.get)(y_val)
y_test_coarse = np.vectorize(dict_coarse.get)(y_test)

y_train_c_cat = np_utils.to_categorical(y_train_coarse)
y_val_c_cat = np_utils.to_categorical(y_val_coarse)
y_test_c_cat= np_utils.to_categorical(y_test_coarse)

In [ ]:
import keras
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Input
from keras import optimizers
from keras.callbacks import TensorBoard, ReduceLROnPlateau, CSVLogger
from keras.layers.normalization import BatchNormalization
from keras.utils.data_utils import get_file
from keras import backend as K

In [ ]:
class TimingCallback(keras.callbacks.Callback):
    
  def on_epoch_begin(self, epoch, logs={}):
    self.starttime=time()
  def on_epoch_end(self, epoch, logs={}):
    logs['time_elapsed'] = (time()-self.starttime)

In [ ]:
class AdaptativeLossWeightsModifier2Vars(keras.callbacks.Callback):
  def __init__(self, alpha, beta, decay_rate=0.5):
    self.alpha = alpha
    self.beta = beta
    self.decay_rate = decay_rate
  
  def calculate_exponential(self, ratio, decay_rate, epoch):
    return np.exp(-ratio*decay_rate*epoch)

  def on_epoch_end(self, epoch, logs={}):
    if epoch < 1:
      pass
    
    else:
      loss_coarse = self.model.history.history['predictions_coarse_loss'][-1]
      loss_fine = self.model.history.history['predictions_fine_loss'][-1]
      ratio = (loss_coarse/loss_fine) 
      decaying = self.calculate_exponential(ratio, self.decay_rate, epoch)
        
      K.set_value(self.alpha, decaying)
      K.set_value(self.beta, 1 - decaying)
      print('Changing loss weights to: coarse = {}, fine = {}'.format(K.eval(self.alpha), K.eval(self.beta)))  
    
    logs['alpha'] = K.eval(self.alpha)
    logs['beta'] = K.eval(self.beta) 

In [ ]:
img_rows, img_cols = 28, 28
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

In [ ]:
num_classes_coarse = 5
num_classes_fine = 10

img_input = Input(shape=input_shape, name='input')


# coarse network
coarse = Conv2D(32, (3, 3), activation='relu', name='coarse_block1_conv1')(img_input)
coarse = MaxPooling2D((2, 2), name='coarse_block1_pool')(coarse)

coarse = Conv2D(64, (3, 3), activation='relu', name='coarse_block1_conv2')(coarse)
coarse = MaxPooling2D((2, 2), name='coarse_block2_pool')(coarse)


coarse = Flatten(name='coarse_flatten')(coarse)
coarse = Dense(128, name='coarse_fc_1')(coarse)
coarse = BatchNormalization()(coarse)
coarse = Activation('relu')(coarse)
coarse = Dropout(0.5)(coarse)

# fine network
fine = Conv2D(32, (3, 3), activation='relu', name='block1_conv1')(img_input)
fine = MaxPooling2D((2, 2), name='block1_pool')(fine)

fine = Conv2D(64, (3, 3), activation='relu', name='block1_conv2')(fine)
fine = MaxPooling2D((2, 2), name='block2_pool')(fine)


fine = Flatten(name='flatten')(fine)
fine = Dense(128, name='fc_1')(fine)
fine = BatchNormalization()(fine)
fine = Activation('relu')(fine)
fine = Dropout(0.5)(fine)

fine = keras.layers.concatenate([fine, coarse])

coarse_pred = Dense(num_classes_coarse, activation='softmax', name='predictions_coarse')(coarse)
fine_pred = Dense(num_classes_fine, activation='softmax', name='predictions_fine')(fine)

In [ ]:
model = Model(inputs=img_input, outputs=[coarse_pred, fine_pred], name='guide_net')

In [41]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
coarse_block1_conv1 (Conv2D)    (None, 26, 26, 32)   320         input[0][0]                      
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 26, 26, 32)   320         input[0][0]                      
__________________________________________________________________________________________________
coarse_block1_pool (MaxPooling2 (None, 13, 13, 32)   0           coarse_block1_conv1[0][0]        
__________________________________________________________________________________________________
block1_poo

In [ ]:
alpha = K.variable(value=0.95, dtype="float32", name="alpha") # A1 in paper
beta = K.variable(value=0.05, dtype="float32", name="beta") # A2 in paper

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              loss_weights=[alpha, beta],
              metrics=['accuracy'])

In [ ]:
tb_cb = TensorBoard(log_dir='drive/TCC-ITAU/fashion-mnist/tensorboard/adaptative-guide-net-simple', histogram_freq=0)
clr_cb = ReduceLROnPlateau(monitor='val_predictions_fine_loss', factor=0.5, patience=10, verbose=1)
time_cb = TimingCallback()
csv_cb = CSVLogger('drive/TCC-ITAU/fashion-mnist/training-data/guide-net/adaptative-guide-net-simple.csv', separator=',', append=False)
change_lw = AdaptativeLossWeightsModifier2Vars(alpha, beta, decay_rate=0.4)

cbks = [tb_cb, clr_cb, time_cb, change_lw, csv_cb]

In [44]:
epochs = 50
start_time = time()
training = model.fit(train_features, [y_train_c_cat, y_train_cat],
                     validation_data=(vali_features, [y_val_c_cat, y_val_cat]),
                     epochs=epochs,
                     batch_size=512, 
                     callbacks=cbks,
                     verbose=1)
training_time = time() - start_time

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 7s 138us/step - loss: 0.2278 - predictions_coarse_loss: 0.2025 - predictions_fine_loss: 0.7074 - predictions_coarse_acc: 0.9339 - predictions_fine_acc: 0.7516 - val_loss: 0.1247 - val_predictions_coarse_loss: 0.1048 - val_predictions_fine_loss: 0.5027 - val_predictions_coarse_acc: 0.9681 - val_predictions_fine_acc: 0.8006
Epoch 2/50
48000/48000 [==============================] - 5s 111us/step - loss: 0.1129 - predictions_coarse_loss: 0.0985 - predictions_fine_loss: 0.3868 - predictions_coarse_acc: 0.9682 - predictions_fine_acc: 0.8602 - val_loss: 0.0943 - val_predictions_coarse_loss: 0.0814 - val_predictions_fine_loss: 0.3408 - val_predictions_coarse_acc: 0.9742 - val_predictions_fine_acc: 0.8771
Changing loss weights to: coarse = 0.8917936682701111, fine = 0.1082063540816307
Epoch 3/50
48000/48000 [==============================] - 5s 112us/step - loss: 0.1079 - predictions_coar

In [45]:
model.evaluate(test_features, [y_test_c_cat, y_test_cat])

10000/10000 [==============================] - 2s 232us/step


[0.3989272073306143, 0.07910943483310985, 0.3989272073306143, 0.9782, 0.9193]

In [46]:
model.save('drive/TCC-ITAU/fashion-mnist/models/adaptative-guide-net-simple.h5')

TypeError: ignored

In [47]:
training_time

297.54555225372314